In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import pickle
import serial
import time

In [2]:
def create_synthetic_dataset():
    """
    Create synthetic dataset similar to real lab sensor data
    Normal ranges:
    - Gas (MQ2): 100-500 ppm
    - Temperature: 20-25°C
    - Humidity: 30-60%
    """
    np.random.seed(42)
    n_samples = 1000
    
    # Normal data
    normal_data = pd.DataFrame({
        'gas_level': np.random.uniform(100, 900, n_samples),
        'temperature': np.random.uniform(20, 30, n_samples),
        'humidity': np.random.uniform(30, 65, n_samples)
    })
    
    # Anomalous data (10% of dataset)
    n_anomalies = n_samples // 10
    anomalous_data = pd.DataFrame({
        'gas_level': np.random.uniform(900, 950, n_anomalies),  # High gas levels
        'temperature': np.random.uniform(30, 40, n_anomalies),   # High temperature
        'humidity': np.random.uniform(65, 80, n_anomalies)       # High humidity
    })
    
    # Combine datasets
    data = pd.concat([normal_data, anomalous_data])
    return data

In [3]:
# 2. Model Training
def train_anomaly_detector(data):
    """Train Isolation Forest model"""
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    
    model = IsolationForest(
        contamination=0.1,  # Expected proportion of anomalies
        random_state=42,
        n_estimators=100
    )
    
    model.fit(scaled_data)
    return model, scaler

In [4]:
# 3. Real-time monitoring with Arduino integration
class LabMonitor:
    def __init__(self, model, scaler, arduino_port='COM3'):
        self.model = model
        self.scaler = scaler
        self.arduino = serial.Serial(port=arduino_port, baudrate=9600)
        time.sleep(2)  # Wait for Arduino connection
        
    def read_sensors(self):
        """Read sensor data from Arduino"""
        if self.arduino.in_waiting:
            try:
                data = self.arduino.readline().decode('utf-8').strip().split(',')
                return {
                    'gas_level': float(data[0]),
                    'temperature': float(data[1]),
                    'humidity': float(data[2])
                }
            except:
                return None
        return None
    
    def check_anomaly(self, sensor_data):
        """Check if current readings are anomalous"""
        data_array = np.array([[
            sensor_data['gas_level'],
            sensor_data['temperature'],
            sensor_data['humidity']
        ]])
        
        scaled_data = self.scaler.transform(data_array)
        prediction = self.model.predict(scaled_data)
        return prediction[0] == -1  # -1 indicates anomaly
    
    def run_monitoring(self):
        """Main monitoring loop"""
        try:
            while True:
                sensor_data = self.read_sensors()
                if sensor_data:
                    is_anomaly = self.check_anomaly(sensor_data)
                    
                    # Send alert signal to Arduino if anomaly detected
                    if is_anomaly:
                        self.arduino.write(b'1')  # Turn on LED
                        print("ALERT: Anomaly detected!")
                        print(f"Readings: {sensor_data}")
                    else:
                        self.arduino.write(b'0')  # Turn off LED
                        
                time.sleep(1)
                
        except KeyboardInterrupt:
            print("Monitoring stopped by user")
            self.arduino.close()

In [5]:
# Main execution
if __name__ == "__main__":
    # Generate and save synthetic dataset
    data = create_synthetic_dataset()
    data.to_csv('lab_sensor_data.csv', index=False)
    
    # Train and save model
    model, scaler = train_anomaly_detector(data)
    pickle.dump(model, open('anomaly_model.pkl', 'wb'))
    pickle.dump(scaler, open('scaler.pkl', 'wb'))
    
    # Start monitoring
    monitor = LabMonitor(model, scaler)
    monitor.run_monitoring()

ALERT: Anomaly detected!
Readings: {'gas_level': 907.0, 'temperature': 37.0, 'humidity': 67.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 907.0, 'temperature': 37.0, 'humidity': 68.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 905.0, 'temperature': 37.0, 'humidity': 67.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 907.0, 'temperature': 37.0, 'humidity': 67.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 907.0, 'temperature': 37.0, 'humidity': 67.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 905.0, 'temperature': 37.0, 'humidity': 67.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 907.0, 'temperature': 37.0, 'humidity': 67.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 915.0, 'temperature': 37.0, 'humidity': 68.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 917.0, 'temperature': 37.0, 'humidity': 68.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 920.0, 'temperature': 37.0, 'humidity': 68.0}
ALERT: Anomaly detected!
Readings: {'gas_level': 9

SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))